In [2]:
import json
import csv
import os
import glob
import pandas as pd

In [3]:
with open("../visData.json") as f:
    data = json.load(f)

In [4]:
file_names =  []

for item in data:
    file_names.append('..' + item['Highlight Data'])

In [5]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')